<a href="https://colab.research.google.com/github/baraa-boujneh/Sales_Predictor/blob/master/Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importing Libraries**

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=40a7c7e5839c8f226d618e20846137ff9f02798f95bf2e3a98b6b3698ff159d2
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
    # EDA Libraries:
    import pandas as pd
    import numpy as np

    import matplotlib.colors as col
    from mpl_toolkits.mplot3d import Axes3D
    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline

    import datetime
    from pathlib import Path
    import random

    # Scikit-Learn models:

    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.ensemble import RandomForestRegressor
    from xgboost.sklearn import XGBRegressor
    from sklearn.model_selection import KFold, cross_val_score, train_test_split

    # LSTM:

    import keras
    from keras.layers import Dense
    from keras.models import Sequential
    from keras.callbacks import EarlyStopping
    import np_utils
    from keras.layers import LSTM


    # ARIMA Model:

    import statsmodels.tsa.api as smt
    import statsmodels.api as sm
    from statsmodels.tools.eval_measures import rmse


    import pickle
    import warnings

**2. Loading and Exploring the Data set**

In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
    warnings.filterwarnings("ignore", category=FutureWarning)
    dataset = pd.read_csv('/content/drive/MyDrive/Sales_Prediction_res/train.csv')
    df_ds = dataset.copy()
    df_ds.head()
    df_ds.info()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Sales_Prediction_res/train.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**3.Data preprocessing**

In [ ]:
#axis=1 means that we will go through each row and delete the columns related to store and item
df_ds=df_ds.drop(['store','item'],axis=1)
#converting date from object datatype to dateTime datatype
df_ds['date']=pd.to_datetime(df_ds['date'])
df_ds.head()


In [ ]:
df_ds['date']=df_ds['date'].dt.to_period("M")
monthly_sales = df_ds.groupby('date').sum().reset_index()
monthly_sales.info()
monthly_sales.head()

In [ ]:
monthly_sales['date']=monthly_sales['date'].dt.to_timestamp()
monthly_sales.info()
monthly_sales.head(10)

**Visualization**

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(monthly_sales['date'],monthly_sales['sales'])
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("monthly customer sales")
plt.show


**call the difference on the sales columns t make the sales data stationary**

In [ ]:
monthly_sales['sales-diff']=monthly_sales['sales'].diff()
monthly_sales=monthly_sales.dropna()
monthly_sales

**Visualization**

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(monthly_sales['date'],monthly_sales['sales-diff'])
plt.xlabel("Date")
plt.ylabel("Sales")
plt.title("monthly customer sales difference")
plt.show


**Dropping off sales and date**

In [ ]:
supervised_data=monthly_sales.drop(['date','sales'], axis=1)
supervised_data

**preparing the supervised data**

In [ ]:
for i in range(1,13):
  col_name='month_'+str(i)
  supervised_data[col_name]=supervised_data['sales-diff'].shift(i)
supervised_data=supervised_data.dropna().reset_index(drop=True)
supervised_data

**Split the data into Train and Test**

In [ ]:
train_data=supervised_data[:-12]
test_data=supervised_data[-12:]
print("Train Data Shape: ", train_data.shape)
print("Test Data Shape: ",test_data.shape)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1,1))
scaler.fit(train_data)
train_data= scaler.transform(train_data)
test_data= scaler.transform(test_data)


In [ ]:
X_train, y_train= train_data[:,1:],train_data[:,0]
X_test, y_test = test_data[:,1:], test_data[:,0]
y_train=y_train.ravel()
y_test=y_test.ravel()
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_Test shape: ", X_test.shape)
print("y_Test shape: ", y_test.shape)

**Make prediction data frame to merge the predicted sales prices of all trained algs**

In [ ]:
sales_dates=monthly_sales['date'][-12:].reset_index(drop=True)
predict_df = pd.DataFrame(sales_dates)

In [ ]:
act_sales = monthly_sales['sales'][-13:].to_list()
print(act_sales)

**To create linear regression model, and predicetd output**

In [ ]:
lr_model=LinearRegression()
lr_model.fit(X_train, y_train)
lr_pre=lr_model.predict(X_test)


In [ ]:
lr_pre=lr_pre.reshape(-1,1)
lr_pre_test_set=np.concatenate([lr_pre, X_test], axis=1)
lr_pre_test_set=scaler.inverse_transform(lr_pre_test_set)
lr_pre_test_set

In [ ]:
result_list=[]
for index in range (0, len(lr_pre_test_set)):
  result_list.append(lr_pre_test_set[index][0]+act_sales[index])
lr_pre_series=pd.Series(result_list, name="Linear Prediction")
predict_df=predict_df.merge(lr_pre_series,left_index=True, right_index=True)
predict_df

In [ ]:
lr_mse= np.sqrt(mean_squared_error(predict_df['Linear Prediction'], monthly_sales['sales'][-12:]))
#MSE and MAE cost function :updating the values of B0 and B1 iteratively until we get an optimal solution.
lr_mae=mean_absolute_error(predict_df['Linear Prediction'], monthly_sales['sales'][-12:])
#R-Squared is a number that explains the amount of variation that is explained/captured by the developed model. It always ranges between 0 & 1 . Overall, the higher the value of R-squared, the better the model fits the
lr_r2=r2_score(predict_df['Linear Prediction'], monthly_sales['sales'][-12:])
print("Linear Regression MSE: ",lr_mse)
print("Linear Regression MAE: ",lr_mae)
print("Linear Regression R2: ",lr_r2)


**Visualization of the prediction against eh actual sales**

In [ ]:
plt.figure(figsize=(15,5))
#Actual sales
plt.plot(monthly_sales['date'], monthly_sales ['sales'])
#Predicted sales
plt.plot(predict_df['date'], predict_df['Linear Prediction'])
plt.xlabel("Date")
plt.ylabel("Sales")
plt.legend(['Actual Sales','Predicted sales'])
plt.show()